In [4]:
file_path = "/content/Harry_Potter_all_books_preprocessed.txt"

with open(file_path, 'r', encoding='utf-8') as file:
    hp_text = file.read()

# Print the first 500 characters to verify the content
print(oz_text[:500])


THE BOY WHO LIVED Mr and Mrs Dursley of number four Privet Drive were proud to say that they were perfectly normal thank you very much .They were the last people youd expect to be involved in anything strange or mysterious because they just didnt hold with such nonsense .Mr Dursley was the director of a firm called Grunnings which made drills .He was a big beefy man with hardly any neck although he did have a very large mustache .Mrs Dursley was thin and blonde and had nearly twice the usual amo


In [ ]:
import numpy as np
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
import tensorflow.keras.utils as keras_utils

# Assuming oz_text contains your text data
sentences = hp_text.split('. ')
tokenizer = Tokenizer(num_words=10000, oov_token="<OOV>")
tokenizer.fit_on_texts(sentences)

sequences = tokenizer.texts_to_sequences(sentences)

# Create input sequences and labels
input_sequences = []
for sequence in sequences:
    for i in range(1, len(sequence)):
        n_gram_sequence = sequence[:i+1]
        input_sequences.append(n_gram_sequence)



In [ ]:
# Pad sequences
max_sequence_len = max([len(x) for x in input_sequences])
input_sequences = np.array(pad_sequences(input_sequences, maxlen=max_sequence_len, padding='pre'))

# Create predictors and labels
train_sequences, labels = input_sequences[:,:-1],input_sequences[:,-1]
vocab_size = len(tokenizer.word_index) + 1  # Adjusting vocab_size here to ensure it's defined before use
labels = keras_utils.to_categorical(labels, num_classes=vocab_size)

# Splitting the data into training and testing sets
train_sequences, test_sequences, train_labels, test_labels = train_test_split(train_sequences, labels, test_size=0.2, random_state=42)

In [3]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense

vocab_size = len(tokenizer.word_index) + 1  # Plus 1 for padding token
embed_dim = 64  # Size of the embedding vector

model = Sequential([
    Embedding(vocab_size, embed_dim, input_length=max_sequence_len-1),
    LSTM(64),
    Dense(vocab_size, activation='softmax')
])


In [5]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Adjust this based on your data preparation step
model.fit(train_sequences, train_labels, epochs=100, validation_data=(test_sequences, test_labels))


Epoch 1/100
1040/1040 [==============================] - 45s 41ms/step - loss: 4.7188 - accuracy: 0.1868 - val_loss: 5.9335 - val_accuracy: 0.1326
Epoch 2/100
1040/1040 [==============================] - 19s 18ms/step - loss: 4.4904 - accuracy: 0.1957 - val_loss: 6.0655 - val_accuracy: 0.1350
Epoch 3/100
1040/1040 [==============================] - 17s 16ms/step - loss: 4.3326 - accuracy: 0.2042 - val_loss: 6.1515 - val_accuracy: 0.1366
Epoch 4/100
1040/1040 [==============================] - 16s 16ms/step - loss: 4.1818 - accuracy: 0.2169 - val_loss: 6.2254 - val_accuracy: 0.1385
Epoch 5/100
1040/1040 [==============================] - 15s 14ms/step - loss: 4.0394 - accuracy: 0.2265 - val_loss: 6.3221 - val_accuracy: 0.1388
Epoch 6/100
1040/1040 [==============================] - 16s 15ms/step - loss: 3.9047 - accuracy: 0.2363 - val_loss: 6.4238 - val_accuracy: 0.1378
Epoch 7/100
1040/1040 [==============================] - 15s 14ms/step - loss: 3.7763 - accuracy: 0.2494 - val_loss: 6

In [6]:
from tensorflow.keras.preprocessing.sequence import pad_sequences

def generate_text(model, tokenizer, start_seed, num_words, max_sequence_len):
    """
    Generate text using a trained model.

    Parameters:
        model: Trained Keras model for text generation.
        tokenizer: Tokenizer used for the model.
        start_seed: Initial seed text to start generation.
        num_words: Number of words to generate.
        max_sequence_len: The maximum length of sequences used by the model.

    Returns:
        A string containing the generated text.
    """
    text_generated = start_seed

    for _ in range(num_words):
        token_list = tokenizer.texts_to_sequences([text_generated])[0]
        token_list = pad_sequences([token_list], maxlen=max_sequence_len-1, padding='pre')
        predicted = np.argmax(model.predict(token_list, verbose=0), axis=-1)

        output_word = ""
        for word, index in tokenizer.word_index.items():
            if index == predicted:
                output_word = word
                break
        text_generated += " " + output_word

    return text_generated


In [22]:
seed_text = "Dorothy"
next_words = 7  # Number of words to generate

# Generate the text
generated_text = generate_text(model, tokenizer, seed_text, next_words, max_sequence_len)

print(generated_text)


Dorothy was captured in the same way and
